<a href="https://colab.research.google.com/github/kameda-yoshinari/IMISToolExeA2021/blob/main/600/pytorch_advanced-revised/3_semantic_segmentation/GC3_8_PSPNet_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.8 推論の実施

- 本ファイルでは、学習させたPSPNetでセマンティックセグメンテーションを行います。


# 学習目標


1.	セマンティックセグメンテーションの推論を実装できるようになる


---

# Google Colab

In [ ]:
!echo "Change to the JST notation."
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime

In [ ]:
!echo "Start mounting your Google Drive."
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
!echo "Move to the working directory."
%cd 202107_Tool-A/Work600/
!ls -l

---
# 共通準備

"pytorch_advanced" folder should be ready before you come here.

In [ ]:
# Skip this if you have already issued git in advance. 
# If you come here by way of 600-PyTorchADL.ipynb, 
# you should skip the git command (as you have already issued in 600).  
# If you run git when pytorch_advanced already exists, git tells the error and clone won't be made.

#!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

import os
if os.path.exists("/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced"):
    print("OK. Alreadly git cloned. You can go.")
else:
    print("You'd better go back to the first 600-PyTorchADL.ipynb")

In [ ]:
!ls

In [ ]:
%cd "pytorch_advanced"

In [ ]:
!ls

In [ ]:
%cd "3_semantic_segmentation"

In [ ]:
!ls

---
# VOC2012準備

VOCdevkit/ from VOCtrainval_11-May-2012.tar is placed at /root .  


In [ ]:
# It takes one minute or so.

%cd /root
!tar xf "/content/drive/My Drive/202107_Tool-A/Work600/pytorch_advanced/2_objectdetection/data/VOCtrainval_11-May-2012.tar"
!ls /root/VOCdevkit
%cd -

In [ ]:
# VOC2012のrootpath
rootpath = "/root/VOCdevkit/VOC2012/"

---
# 事前準備

- 学習させた重みパラメータ「pspnet50_30.pth」をフォルダ「weights」に用意してあるものとする。

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch

# ファイルパスリストを用意

In [ ]:
from utils.dataloader import make_datapath_list, DataTransform


# ファイルパスリスト作成
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

# 後ほどアノテーション画像のみを使用する


# ネットワークを用意

In [ ]:
from utils.pspnet import PSPNet

net = PSPNet(n_classes=21)

# 学習済みパラメータをロード
state_dict = torch.load("./weights/pspnet50_30.pth",
                        map_location={'cuda:0': 'cpu'})
net.load_state_dict(state_dict)

print('ネットワーク設定完了：学習済みの重みをロードしました')


# 推論実行

In [ ]:
# Just to surpress UserWarning
import warnings
warnings.simplefilter('ignore')

# 1. 元画像の表示
image_file_path = "./data/cowboy-757575_640.jpg"
img = Image.open(image_file_path)   # [高さ][幅][色RGB]
img_width, img_height = img.size
plt.imshow(img)
plt.show()

# 2. 前処理クラスの作成
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)
transform = DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std)

# 3. 前処理
# 適当なアノテーション画像を用意し、さらにカラーパレットの情報を抜き出す
anno_file_path = val_anno_list[0]
anno_class_img = Image.open(anno_file_path)   # [高さ][幅]
p_palette = anno_class_img.getpalette()
phase = "val"
img, anno_class_img = transform(phase, img, anno_class_img)


# 4. PSPNetで推論する
net.eval()
x = img.unsqueeze(0)  # ミニバッチ化：torch.Size([1, 3, 475, 475])
outputs = net(x)
y = outputs[0]  # AuxLoss側は無視 yのサイズはtorch.Size([1, 21, 475, 475])


# 5. PSPNetの出力から最大クラスを求め、カラーパレット形式にし、画像サイズを元に戻す
y = y[0].detach().numpy()  # y：torch.Size([1, 21, 475, 475])
y = np.argmax(y, axis=0)
anno_class_img = Image.fromarray(np.uint8(y), mode="P")
anno_class_img = anno_class_img.resize((img_width, img_height), Image.NEAREST)
anno_class_img.putpalette(p_palette)
plt.imshow(anno_class_img)
plt.show()


# 6. 画像を透過させて重ねる
trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBAに変換

for x in range(img_width):
    for y in range(img_height):
        # 推論結果画像のピクセルデータを取得
        pixel = anno_class_img.getpixel((x, y))
        r, g, b, a = pixel

        # (0, 0, 0)の背景ならそのままにして透過させる
        if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
            continue
        else:
            # それ以外の色は用意した画像にピクセルを書き込む
            trans_img.putpixel((x, y), (r, g, b, 150))
            # 150は透過度の大きさを指定している

img = Image.open(image_file_path)   # [高さ][幅][色RGB]
result = Image.alpha_composite(img.convert('RGBA'), trans_img)
plt.imshow(result)
plt.show()


以上

---
Revised by KAMEDA, Yoshinari at University of Tsukuba for lecture purpose.  
Original: https://github.com/YutaroOgawa/pytorch_advanced

2021/08/02. 